In [15]:
import pyodbc
import reddit_engine as rpe 
import importlib
import time

# Force le rechargement du module .py si vous faites des changements
importlib.reload(rpe)

<module 'reddit_engine' from 'c:\\Users\\samaz\\Desktop\\TFF\\MarketReviews_VS_TRUTH\\reddit_engine.py'>

In [16]:
print("--- Lancement de l'Extraction (Étape 6a) ---")
print("Objectif : Lire Dim_Product depuis le DWH.")

conn_dwh = None
cursor_dwh = None
product_list_to_search = [] # La variable qui stockera notre liste de travail

try:
    conn_dwh = rpe.get_dwh_connection()
    cursor_dwh = conn_dwh.cursor()
    print("   -> ✅ Connecté au DWH.")

    # Appelle la fonction de votre .py
    product_list_to_search = rpe.get_products_to_search(cursor_dwh)
    
    if product_list_to_search:
        print("\n--- ✅ SUCCÈS : Liste des produits à rechercher extraite ---")
        print(f"{len(product_list_to_search)} produits au total.")
        print("Voici les 5 premiers produits :")
        for product in product_list_to_search[:5]:
            print(f"   - Key: {product.ProductKey}, Nom: {product.ProductName}")
    else:
        print("\n--- ⚠️ AVERTISSEMENT : Aucun produit trouvé dans Dim_Product. ---")

except Exception as e:
    print(f"\n--- ❌ ERREUR : {e} ---")
finally:
    if cursor_dwh: cursor_dwh.close()
    if conn_dwh: conn_dwh.close()
    print("\nConnexion DWH fermée. Script d'extraction terminé.")

--- Lancement de l'Extraction (Étape 6a) ---
Objectif : Lire Dim_Product depuis le DWH.
   -> ✅ Connecté au DWH.
   -> (E) Extraction des produits depuis Dim_Product...
   -> 1296 produits trouvés à rechercher sur Reddit.

--- ✅ SUCCÈS : Liste des produits à rechercher extraite ---
1296 produits au total.
Voici les 5 premiers produits :
   - Key: 477, Nom: JBL PARTYBOX ULTIMATE
   - Key: 478, Nom: SONOS MOVE 2 WHITE
   - Key: 479, Nom: JBL BOOMBOX 3 BLACK
   - Key: 480, Nom: MARSHALL  ACTON III  BT BLACK
   - Key: 481, Nom: BOSE SOUNDLINK MICRO BLACK

Connexion DWH fermée. Script d'extraction terminé.


In [17]:
print("--- Lancement du Scraping/Chargement (Étape 6b & 6c) ---")

if 'product_list_to_search' not in locals() or not product_list_to_search:
    print("--- ❌ ERREUR : La variable 'product_list_to_search' est vide. ---")
    print("   Veuillez (ré)exécuter la cellule précédente pour extraire la liste des produits.")
else:
    
    product_list_test = product_list_to_search
    print(f"\n--- Lancement de la boucle de scraping pour {len(product_list_test)} produits (TEST) ---")

    for product in product_list_test:
        product_key = product.ProductKey
        product_name = product.ProductName # On récupère le nom
        brand = product.Brand
        
        conn_staging = None
        cursor_staging = None
        
        print(f"\n-------------------------------------------------")
        print(f"--- Traitement du Produit {product_key} : '{product_name}' ---")
        
        try:
            search_keyword = rpe.generate_search_keyword(product_name, brand)
            print(f"   -> Mot-clé généré : '{search_keyword}'")
            
            if not search_keyword:
                print("   -> Mot-clé vide, impossible de chercher. On ignore.")
                continue

            raw_posts = rpe.search_reddit_api(search_keyword, subreddit="headphones")
            
            if raw_posts:
                conn_staging = rpe.get_staging_connection()
                cursor_staging = conn_staging.cursor()
                
                # --- APPEL MODIFIÉ ---
                # On passe maintenant les 2 nouvelles variables
                rpe.save_raw_posts_to_staging(
                    cursor_staging, 
                    raw_posts, 
                    product_key, 
                    product_name,
                    search_keyword    
                )
                
                conn_staging.commit()
                print(f"   -> Données pour '{product_name}' sauvegardées.")
            else:
                print(f"   -> Aucun post trouvé pour '{search_keyword}'. On continue.")

            print("   -> Pause de 3 secondes...")
            time.sleep(7) 

        except Exception as e:
            print(f"   ->  ERREUR lors du traitement du SKU {product_name} : {e}")
            if conn_staging: conn_staging.rollback()
        finally:
            if cursor_staging: cursor_staging.close()
            if conn_staging: conn_staging.close()

    print("\n---  Boucle de scraping Reddit terminée. ---")

--- Lancement du Scraping/Chargement (Étape 6b & 6c) ---

--- Lancement de la boucle de scraping pour 1296 produits (TEST) ---

-------------------------------------------------
--- Traitement du Produit 477 : 'JBL PARTYBOX ULTIMATE' ---
   -> Mot-clé généré : 'JBL PARTYBOX ULTIMATE'
   -> (E) Appel API Reddit pour : 'JBL PARTYBOX ULTIMATE' dans r/headphones
   -> 1 posts bruts trouvés.
   -> (L) Chargement de 1 posts dans Staging_Reddit_Posts...
   -> 0 nouveaux posts insérés.
   -> 1 posts ignorés (doublons).
   -> Données pour 'JBL PARTYBOX ULTIMATE' sauvegardées.
   -> Pause de 3 secondes...

-------------------------------------------------
--- Traitement du Produit 478 : 'SONOS MOVE 2 WHITE' ---
   -> Mot-clé généré : 'SONOS MOVE 2'
   -> (E) Appel API Reddit pour : 'SONOS MOVE 2' dans r/headphones
   -> 25 posts bruts trouvés.
   -> (L) Chargement de 25 posts dans Staging_Reddit_Posts...
   -> 0 nouveaux posts insérés.
   -> 25 posts ignorés (doublons).
   -> Données pour 'SONOS